# Esperimento 1 - Deep Learning


---

**Esperimento:** riconoscere simmetrie su un array tramite l'uso di un'architettura neurale, composta da un hidden layer con 2 neuroni e un output layer con un neurone.

### Inizializzazione parametri
Iniziamo importando le librerie necessarie e definendo i parametri:

In [359]:
import torch
import numpy as np

learning_rate = 0.1
momentum = 0.95
array_size = 6    # E' possibile selezionare arrays di dimensione più grande, nel qual caso l'architettura si adatterà automaticamente.
gamma = 1.0       # Nel caso si voglia impostare un learning rate che diminuisce progressivamente.

### Funzioni
Definiamo una funzione che restituisce 1 se l'array è simmetrico, 0 altrimenti. Essa verrà eseguita sul dataset per ottenere i labels:

In [360]:
def isSymmetric(a):
    for idx in range(int(len(a)/2)):
        if a[idx]!=a[len(a)-idx-1]:
            return 0
    return 1

Definiamo una funzione capace di generare tutte le 64 (2^n) combinazioni di array di lunghezza 6 (n) composte da 0 e 1:

In [361]:
def cartesian_product(*arrays):
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    return points

perms = cartesian_product(*array_size*[np.arange(2)])

### Inizializzazione dataset e architettura
Andiamo a definire il nostro dataset: esso è composto da array di valore 0,1 di lunghezza 6 (n).

In [362]:
batch_size = 8
torch.manual_seed(0)

X = torch.tensor(np.array(np.meshgrid(perms)).T.reshape(-1, array_size)).float()
Y = torch.tensor(np.apply_along_axis(isSymmetric, 1, X)).float()    # Applichiamo la funzione isSymmetric al dataset per ottenere le labels

print(X)
print(Y)

dataset = torch.utils.data.TensorDataset(X,Y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 1., 1.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 0., 1.],
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1., 1.],
        [1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 1.],
        [1., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 1., 1.],
        [1., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1.],
        [1., 0., 1., 0., 1., 0.],
        [1., 0., 1., 0., 1., 1.],
        [1., 0., 1., 1., 0., 0.],
        [1., 0

Definiamo il modello da utilizzare, inizializzando i pesi tra -0.3 e 0.3:

In [363]:
np.random.seed(1)

class Model(torch.nn.Module):
    def __init__(self, inputs=6):
        super().__init__()
        self.layer = torch.nn.Linear(in_features=inputs, out_features=2, bias=True)
        self.out = torch.nn.Linear(in_features=2, out_features=1, bias=True)
        self.activation = torch.nn.Sigmoid()

        # Inizializzazione pesi
        self.layer.weight.data=torch.tensor((np.random.uniform(low = -0.3, high = 0.3, size=(2,inputs)))).float()
        self.out.weight.data=torch.tensor((np.random.uniform(low = -0.3, high = 0.3, size=(1,2)))).float()
        
    def forward(self, x):
        x=self.activation(self.layer(x))
        x=self.activation(self.out(x))
        return x


model = Model(inputs=array_size)

print(model.layer.weight)
print(model.out.weight)

Parameter containing:
tensor([[-0.0498,  0.1322, -0.2999, -0.1186, -0.2119, -0.2446],
        [-0.1882, -0.0927, -0.0619,  0.0233, -0.0485,  0.1111]],
       requires_grad=True)
Parameter containing:
tensor([[-0.1773,  0.2269]], requires_grad=True)


Definiamo l'optimizer, lo scheduler e la funzione loss:

In [364]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
loss = torch.nn.MSELoss()

### Allenamento architettura
Alleniamo quindi l'architettura sul dataset:

In [365]:
epochs = 2000
losses = []
for epoch in range(epochs):
    epoch_loss=0
    model.train()
    if int(epoch%(epochs/10)) == 0:
        print("Epoch: ", epoch)
    for x, y in iter(dataloader):
        out=model(x)
        l=loss(out, y.unsqueeze(1))
        epoch_loss+=l.item()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    losses.append(epoch_loss/len(dataloader))
    if int(epoch%(epochs/10)) == 0:
        print("Loss: ", losses[epoch])
    scheduler.step()

print(np.array(losses))
print(model.layer.weight)
print(model.layer.bias)
print(model.out.weight)
print(model.out.bias)

Epoch:  0
Loss:  0.267031654715538
Epoch:  200
Loss:  0.10921449877787381
Epoch:  400
Loss:  0.09309924021363258
Epoch:  600
Loss:  0.01585260080082662
Epoch:  800
Loss:  0.0028608086504391395
Epoch:  1000
Loss:  0.0012804762382074841
Epoch:  1200
Loss:  0.0007857828113628784
Epoch:  1400
Loss:  0.0005556969728601757
Epoch:  1600
Loss:  0.0004234138199166182
Epoch:  1800
Loss:  0.0003399314450440727
[0.26703165 0.13060955 0.10662537 ... 0.00028323 0.00028331 0.00028341]
Parameter containing:
tensor([[ 11.5517,   5.7982,  -2.9075,   2.9054,  -5.8000, -11.5517],
        [-11.5520,  -5.7996,   2.9057,  -2.9077,   5.7985,  11.5517]],
       requires_grad=True)
Parameter containing:
tensor([[-14.6746, -14.6748]], requires_grad=True)


### Osservazioni

I pesi ottenuti dall'architettura possiedono una simmetria analoga a quella presentata nell'articolo (i pesi simmetrici rispetto alla metà tendono ad avere modulo uguale e segno opposto, con i pesi del neurone di output negativi).<br><br>
E' stato anche osservato che la quantità di epoche richieste per raggiungere questa particolare simmetria varia molto in base all'inizializzazione dei pesi dell'architettura (è stato necessario forzare il seed per ottenere un risultato consistente), e che la funzione loss tende a ristagnare su un valore di ~0.1 per gli array di dimensione 6, probabilmente a causa di un minimo locale. Questo fenomeno è osservabile su array di ogni dimensione, a valori di loss diversi. La causa è molto probabilmente dovuta al fatto che la maggior parte degli array di lunghezza 6 non sono simmetrici, portando così la funzione verso un'approssimazione sbagliata a 0.<br><br>
